In [24]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install netCDF4
from netCDF4 import Dataset
import datetime as dt
import numpy as np
import pandas as pd
import os

In [27]:
InputLocation = 'C:\\Users\\00065525\\AED Dropbox\\Scratch\\Hawkesbury\\Cattai\\Matlab\\MatlabScriptsToUWA\\TUFLOW FV NetCDF to CSV Alternate Method\\Profiles_2\\'
OutputLocation = 'C:\\Users\\00065525\\AED Dropbox\\Scratch\\Hawkesbury\\Cattai\\Matlab\\MatlabScriptsToUWA\\TUFLOW FV NetCDF to CSV Alternate Method\\Profiles_2\\'
SiteDetailsFile = 'NamePTIndex_v2.csv'
ParamDetailsFile = 'Variables.csv'
startDate = dt.datetime(1990,1,1,0,0,0,0)

In [28]:
SiteFile = os.path.join(InputLocation,SiteDetailsFile)
siteData = pd.read_csv(SiteFile)
ParamFile = os.path.join(InputLocation,ParamDetailsFile)
paramData = pd.read_csv(ParamFile)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\00065525\\AED Dropbox\\Scratch\\Hawkesbury\\Cattai\\Matlab\\MatlabScriptsToUWA\\TUFLOW FV NetCDF to CSV Alternate Method\\Profiles_2\\Variables.csv'

In [29]:
ncFile = os.path.join(InputLocation,"HN_Cal_2017_2018_2D_wq_DOC_DEM_3D_WQ_1_New_PROFILES.nc")
tempDataset = Dataset(ncFile,'r')

FileNotFoundError: [Errno 2] No such file or directory: b'C:\\Users\\00065525\\AED Dropbox\\Scratch\\Hawkesbury\\Cattai\\Matlab\\MatlabScriptsToUWA\\TUFLOW FV NetCDF to CSV Alternate Method\\Profiles_2\\HN_Cal_2017_2018_2D_wq_DOC_DEM_3D_WQ_1_New_PROFILES.nc'

In [30]:
print(tempDataset['Ocean'])

<class 'netCDF4._netCDF4.Group'>
group /Ocean:
    dimensions(sizes): NumLayers(9), NumLayerFaces(10), N1(1)
    variables(dimensions): float64 X(N1), float64 Y(N1), float32 layerface_Z(Time, NumLayerFaces), float32 D(Time, N1), float32 H(Time, N1), float32 SAL(Time, NumLayers), float32 TEMP(Time, NumLayers), float32 TSS(Time, NumLayers), float32 V_x(Time, NumLayers), float32 V_y(Time, NumLayers), float32 WQ_DIAG_PHY_TCHLA(Time, NumLayers), float32 WQ_DIAG_TOT_TN(Time, NumLayers), float32 WQ_DIAG_TOT_TOC(Time, NumLayers), float32 WQ_DIAG_TOT_TP(Time, NumLayers), float32 WQ_DIAG_TOT_TSS(Time, NumLayers), float32 WQ_DIAG_TOT_TURBIDITY(Time, NumLayers), float32 WQ_NCS_SS1(Time, NumLayers), float32 WQ_NIT_AMM(Time, NumLayers), float32 WQ_NIT_NIT(Time, NumLayers), float32 WQ_OGM_DOC(Time, NumLayers), float32 WQ_OGM_DON(Time, NumLayers), float32 WQ_OGM_DOP(Time, NumLayers), float32 WQ_OGM_POC(Time, NumLayers), float32 WQ_OGM_PON(Time, NumLayers), float32 WQ_OGM_POP(Time, NumLayers), float32 

In [31]:
for subdir, dirs, files in os.walk(InputLocation):
    for file in files:
        filepath = subdir + os.sep + file
        if not filepath.endswith(".nc"):
            continue
        
        print (filepath)

        outFile = filepath[:-2] + 'csv'
        tuflowDataset = Dataset(filepath,'r')
        times = tuflowDataset.variables['ResTime'][:]
        timesArray = np.ma.filled(times,np.nan)
        timesArray=timesArray.round(0)
        actualDateList = [startDate + dt.timedelta(hours=t) for t in timesArray]
        Results = pd.DataFrame()
        Results['TIME']=actualDateList
        Results = Results.set_index('TIME')
        for index, row in siteData.iterrows():
            tsInName = row['Name']
            tsOutName = row['Pt']
            for index2,row2 in paramData.iterrows():
                thisTs = []
                thisTs = tuflowDataset[tsInName][row2['Variable']][:]
                if len(thisTs[0])>1:
                    thisTs = thisTs.mean(axis=1)

                Results[tsOutName + '_' + row2['Variable']] = thisTs
        
        Results.to_csv(outFile,index=True,date_format='%d-%m-%Y %H:%M')
  

C:\Users\00065525\AED Dropbox\Scratch\Hawkesbury\Cattai\Matlab\MatlabScriptsToUWA\TUFLOW FV NetCDF to CSV Alternate Method\Profiles_2\\HN_Cal_2017_2018_2D_wq_DOC_DEM_3D_WQ__New_PROFILES.nc
